By inferring, we mean the model takes a text as input and performs some sort of anlysis, such as extracting sentiment or names. In traditional ML work, you would have to collect examples with labels and train. But with LLMs, we can just write a prompt, and work with one model/API.

In this example, first we work with a product review, and write prompts to figure out the sentiment and emotions, and extract product information. Then, we work with a news article and infer the topics.

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def get_completion(prompt, model="gpt-4", temp=0):
    """
    Generate a text completion using the OpenAI API.

    Parameters:
        prompt (str): The user's prompt or input text.
        model (str, optional): The model to use (default is "gpt-4").
        temp (float, optional): The temperature parameter controlling randomness (default is 0).

    Returns:
        str: The generated text completion.
    """
    messages = [
        {"role": "user", 
         "content": prompt}
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp  
    )
    return response.choices[0].message.content

In [3]:
product_review = f"""
The positive thing is the size. But it lacks the power/heat :-(. Needs to be 500w/1800+ . 
I noticed that fries and other items take longer in this fryer/not as crispy. It also lacks 
features that other fryers have .
- No mid time flip notification
- when time is finished it just shuts down / my other air fryer had a cooling time when timer 
finished. Meaning the internal fan would keep running until the unit cooled down/this one does 
Not have that.
I would probably prefer my old Big Box air fryers and give up the size of this one.
"""

In [4]:
# infer sentiment
prompt1 = f"""
What is the sentiment of the following product review, which is delimted with triple backticks?

Review text: ```{product_review}```
"""

response = get_completion(prompt1)
print(response)

The sentiment of the product review is negative.


In [5]:
# infer sentiment, and specify its format: one word (positive or negative)
prompt2 = f"""
What is the sentiment of the following product review, which is delimted with triple backticks?

Give one word response "positive" or "negative".

Review text: ```{product_review}```
"""

response = get_completion(prompt2)
print(response)

negative


In [6]:
# infer emotions in the text
prompt3 = f"""
Identify a list of emotions that the writer of the following review is expressing. 
Include no more than five items in the list. 
Format your answer as a list of lower-case words seperated by commas.
The review is delimted with triple backticks.

Review text: ```{product_review}```
"""

response = get_completion(prompt3)
print(response)

disappointment, frustration, dissatisfaction, regret, sadness


In [7]:
# check if a particular emotion (e.g., anger) is present
prompt4 = f"""
Is the author of the following review expressing anger?
Give your answer as either yes or no.
The review is delimted with triple backticks.

Review text: ```{product_review}```
"""

response = get_completion(prompt4)
print(response)


No


In [8]:
# extract product information
prompt6 = f"""
Identify the following items from the review text:
- Product purchased by reviewer
- Company that made the product
The review id delimted with triple backticks. 
Formate the response as JSON objec with "Item" and "Brand" as keyes.
If the information isn't present, use "unknown" as value.
Make the response as sort as possible.

Review text: ```{product_review}```
"""

response = get_completion(prompt6)
print(response)

{"Item": "air fryer", "Brand": "unknown"}


In [9]:
# Do it all in one prompt!
prompt7 = f"""
Identify the following items from the review text:
1. Sentiment(one word answer: positive or negative)
2. Is the reviewer expressing anger? (boolean answer: true or false)
3. Product purchased by the reviewer
4. Company that made the product

The reviewer is delimited with triple backticks.
Format your response as a JSON object with the following keys:
"Sentiment", "Anger", "Product", "Brand".
If the information is not present, use "unknown".

Make your response as short as possible.

Review text: ```{product_review}```
"""
response = get_completion(prompt7)
print(response)

{"Sentiment": "negative", "Anger": false, "Product": "air fryer", "Brand": "unknown"}


In [10]:
# Infer the topics of a news article
# https://decrypt.co/212793/sam-altman-openai-bill-gates-microsoft
news_article = f"""
Sam Altman, securely back at the help of leading global artificial intelligence firm OpenAI, 
sees largely positive trade offs to technology that is racing quickly to match human intelligence.

"Although we are giving something up here, in some sense we are going to have things that are smarter 
than us," he told tech mogul Bill Gates during a recent podcast conversation, “If we can get into this 
world of post scarcity, we will find new things to do.”

Altman and Gates engaged in an insightful dialogue that netted profound insights, peeling back the 
layers of Altman's perspectives on AI and its trajectory. But first, Altman sought to reassure 
industry watchers following the ultimately unsuccessful corporate coup last year.

“A lot of people have remarked on the fact that the team has never felt more productive, or more 
optimistic, or better," Altman said.

Altman touched upon the philosophical aspects of AI, contemplating a future where AI surpasses human 
intelligence. He has previously shared his concerns regarding the socioeconomic impact of AI, calling 
for better regulations that ensure a properly aligned AI development.

For Altman, AI will lead to a society in which workers will be able to do more things for the same amount 
of money, making them more productive for their employers.

"If you make a programmer three times more effective, it's not just that they can write, they can do three 
times more stuff, it's that they can... think of totally different things," he said.

For Altman, this shift in the work dynamics that AI is causing is inevitable.

“The part that I find potentially a little scary is just the speed with which society is going to have to 
adapt and that the labor market will change,” he said.

Altman previously said that “the hypothetical idea that we already have done something really bad by 
launching ChatGPT” is something that bothers him. However, as worried as he may sound for the future of 
those affected by AI, OpenAI is relentlessly pushing the boundaries of its GPT models, presenting more 
powerful LLMs, a store for customized agents that could easily replace more jobs, and partnering with news 
sites to train its future GPT-5 model on their content.

As for the future of AI, Altman emphasized the need for significant leaps in AI's cognitive skills. For him, 
multimodality (the capacity of a model to understand inputs that go beyond text, and contain images or videos 
for example) will play a key role in determining which model dominates the AI race

However, the quality of the outputs will be the differentiating factor, and “the most important areas of progress 
will be around reasoning ability... and also reliability,” he said.

Altman and Gates discussed the topic of regulatory and ethical constraints, exploring calls for a cohesive global 
regulatory framework for AI. Considering the expansive impact of advanced AI systems, he once again advocated for 
a balanced, international governance approach:

"For these... future extraordinarily powerful systems, we have been socializing the idea of a global regulatory 
body," he said.

Altman's reflections present a multifaceted outlook on AI's future: a mix of hope and prudence, ingenuity and 
accountability. New players in the AI field are challenging OpenAI’s dominance, and regulators (just as everyday 
workers) are more worried than ever.

That doesn’t scare Alman.

"It's, you know, both annoying, motivating and fun," he said, “But it does push us to be better and do faster 
and do things faster.”
"""

In [11]:
prompt8 = f"""
Determine five topics that are being discussed in the following text, 
which is delimited with triple backticks.

Make each item one or two words long.

Format your response as a list of items seperated by commas.

Text: ```{news_article}```
"""

response = get_completion(prompt8)
print(response)
# process as a python list
print(response.split(","))

Artificial Intelligence, AI Development, Socioeconomic Impact, AI Regulation, Work Dynamics
['Artificial Intelligence', ' AI Development', ' Socioeconomic Impact', ' AI Regulation', ' Work Dynamics']


In [12]:
# Is a particular topic (or a few) present?
# Example of zero-shot learning (we did not provide training/labeled data for this classification task!)
topic_list = ["AI", "Job Satisfaction", "Technology", "Global Economy", "Federal Government"]
prompt9 = f"""Determine whether each item in the following list of topics 
is a topic discussed in the text below. 

The text is delimited with triple backticks.

The list of topics is {", ".join(topic_list)}

Text: ```{news_article}```

Format your response as a JSON, where the keys are the topics, 
and the values are boolean (true or false) values indicating whether 
that topic is discussed in the text.
"""

response = get_completion(prompt9)
print(response)


{
  "AI": true,
  "Job Satisfaction": false,
  "Technology": true,
  "Global Economy": false,
  "Federal Government": false
}
